In [4]:
import sys, os
import pandas as pd
sys.path.append('../src/')
from Biologging_Toolkit.applications.Jerk import Jerk

/home6/grosmaan/Documents/SES_tags/notebooks/../src/Biologging_Toolkit


#### Enter deployment ID and path

Finalized dataset will be saved or loaded from path

In [5]:
depid = 'ml17_280a'
path = '/run/media/grosmaan/LaCie/individus_brut/'
sens_path = os.path.join(path, 'CTD', depid, depid+'sens5.nc')
raw_path = os.path.join(path, 'individus', depid, 'raw')

#### Run cells to get acoustic features

Time can be constructed from extisting time array or by entering the path to sens5 dataset.

In [6]:
inst = Jerk(depid, 
            path = os.path.join(path, 'individus',depid), 
            sens_path = sens_path, 
            raw_path = raw_path,
           )

In [7]:
inst()

In [8]:
inst.ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: Processed dataset for ml17_280a
    subtitle: NetCDF structure storing processed data from the ml17_280a individual using a 3 s timestep
    sampling_rate: 3
    sampling_rate_units: Seconds
    dimensions(sizes): time(528420)
    variables(dimensions): float64 time(time), float32 depth(time), float64 dives(time), float32 lat(time), float32 lon(time), float64 azimuth(time), float64 elevation_angle(time), float64 bank_angle(time), float64 vertical_azimuth(time), float64 inertial_speed(time), float64 jerk(time)
    groups: 

In [9]:
inst.ds['jerk']

<class 'netCDF4._netCDF4.Variable'>
float64 jerk(time)
    units: m.s**2
    long_name: high resolution jerks
    raw_data_samplerate: 200
    threshold: 400
    threshold_units: m.s**2
    blanking: 0.25
    blanking_units: s
    duration: 0.02
    duration_comment: Maximum duration of peaks
    duration_units: s
unlimited dimensions: 
current shape = (528420,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [10]:
inst.ds.close()